<a href="https://colab.research.google.com/github/Sandhya-Budili/Generative-AI-Tasks/blob/main/Memory_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q sentence-transformers langchain chromadb pypdf faiss-cpu \
langchain_community scikit-learn numpy mistralai langchain-mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.2/328.2 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 79.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 461.0/461.0 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.7 MB/s eta 

In [5]:
from langchain_mistralai import ChatMistralAI, MistralAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain_core.prompts import PromptTemplate
from langchain_classic.memory import ConversationBufferMemory
from langchain_classic.chains import ConversationalRetrievalChain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader

import os
from getpass import getpass

In [6]:
os.environ["MISTRAL_API_KEY"] = getpass("Enter Mistral API Key: ")

Enter Mistral API Key: ··········


In [9]:
!pip install pypdf

In [10]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/Artificial Intelligence.pdf")   # upload your PDF to colab
docs = loader.load()
len(docs)

3

In [11]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)

chunks = text_splitter.split_documents(docs)
len(chunks)


7

In [12]:
embeddings = MistralAIEmbeddings(model="mistral-embed")

vector_db = FAISS.from_documents(chunks, embedding=embeddings)
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer.json: 0.00B [00:00, ?B/s]

In [13]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

/tmp/ipython-input-515166408.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [14]:
template = """
You are a Memory-Augmented RAG assistant.

Conversation History:
{chat_history}

Relevant Retrieved Context:
{context}

User Query:
{question}

Provide a context-aware answer. ALWAYS use history if relevant.
"""

prompt = PromptTemplate(
    input_variables=["chat_history", "context", "question"],
    template=template
)

In [15]:
llm = ChatMistralAI(model="mistral-large-latest", temperature=0)

rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    combine_docs_chain_kwargs={"prompt": prompt}
)

In [ ]:
while True:
    q = input("You: ")
    if q.lower() in ["exit", "quit"]:
        break
    resp = rag_chain.invoke({"question": q})
    print("Bot:", resp["answer"])

You: what is this file about
Bot: Here’s a **context-aware, structured answer** that synthesizes the retrieved context, our conversation history, and your standalone query about the content/purpose of the AI file:

---

### **Content of the File: A Deep Dive**
The file is a **comprehensive primer on Artificial Intelligence (AI)**, organized into **8 key sections** that collectively explain:
1. **AI Fundamentals** (Definition, goals, and evolution from automation to adaptive systems).
2. **Core Technologies**:
   - **Machine Learning (ML)**: How systems learn from data (supervised/unsupervised/reinforcement learning) with examples like retail recommendations.
   - **Deep Learning (DL)**: Neural networks powering breakthroughs in computer vision, speech recognition, and **Large Language Models (LLMs)**.
3. **Advanced Techniques**:
   - **Retrieval-Augmented Generation (RAG)**: A hybrid method combining **external data retrieval** (e.g., databases, APIs) with LLM generation to improve acc